New type test - check and coerce data to a custom type and add new validators to that type

In [1]:
from collections import namedtuple

import valideer
from valideer import ValidationError

In [2]:
Dimensions = namedtuple("Dimensions", "length, width, height")

In [3]:
class MaxDimensionLength(valideer.Validator):
    def __init__(self, max_dim_length):
        self.max_dim_length = max_dim_length
    
    def validate(self, value, adapt=True):
        if max(value) > self.max_dim_length:
            raise ValidationError(f"largest dimension is greater than {self.max_dim_length}", value)
        return value

In [4]:
class MaxVolume(valideer.Validator):
    def __init__(self, max_volume):
        self.max_volume = max_volume
        
    def validate(self, value, adapt=True):
        vol = value.length * value.width * value.height
        if vol > self.max_volume:
            raise ValidationError(f'total volume is greaten than {self.max_volume}', value)
        return value

In [5]:
schema = {
    "size": valideer.ChainOf(
        valideer.AdaptBy(Dimensions._make),
        valideer.AllOf(
            MaxDimensionLength(10),
            MaxVolume(250)
        )
    )
}
validator = valideer.parse(schema)

In [6]:
assert validator.is_valid({
    'size': Dimensions(8, 5, 6)
})

In [7]:
validator.validate({
    'size': (8, 5, 3)
})

{'size': Dimensions(length=8, width=5, height=3)}

In [8]:
try:
    validator.validate({
        'size': (11, 0.5, 0.3)
    })
except ValidationError as e:
    print(e.message)

Invalid value Dimensions(length=11, width=0.5, height=0.3) (Dimensions): largest dimension is greater than 10 (at size)


In [9]:
try:
    validator.validate({
        'size': (8, 5, 7)
    })
except ValidationError as e:
    print(e.message)

Invalid value Dimensions(length=8, width=5, height=7) (Dimensions): total volume is greaten than 250 (at size)
